In [10]:
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv("../data_Kaggle/train.csv")

In [3]:
train

,Insult,Date,Comment
0,1,20120618192155Z,"""You fuck your dad."""
1,0,20120528192215Z,"""i really don't understand your point.\xa0 It ..."
2,0,NaN,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,0,NaN,"""listen if you dont wanna get married to a man..."
4,0,20120619094753Z,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."
...,...,...,...
3942,1,20120502172717Z,"""you are both morons and that is never happening"""
3943,0,20120528164814Z,"""Many toolbars include spell check, like Yahoo..."
3944,0,20120620142813Z,"""@LambeauOrWrigley\xa0\xa0@K.Moss\xa0\nSioux F..."
3945,0,20120528205648Z,"""How about Felix? He is sure turning into one ..."


In [4]:
def clean_df(df_base, column):
    '''Cleaning column 'text' of a dataframe. Remove '@', links, punctuation and stopwords. Also tokenize the string

    Params:
        df - dataframe to be cleaned
    Returns:
        Cleaned dataframe
    '''
    
    df = df_base.copy(deep=True)
    #Remove @names
    df['clean'] = df[column].str.replace(r'@[A-Za-z0-9_]+', '', regex=True)
    df['clean'] = df['clean'].str.replace(r'http\S+', '', regex=True)
    
    #Remove space
    df['clean'] = df['clean'].map(lambda x: re.sub('\n', ' ', x))
    
    #Remove numbers
    df['clean'] = df['clean'].map(lambda x: re.sub(r'\d', '', x))
    
    # Remove punctuation
    df['clean'] = df['clean'].map(lambda x: re.sub('[,\.!?]"', ' ', x))
    #df['text_clean2'] = df['text_clean2'].map(lambda x: re.sub('@', '', x))
    
    # Convert the titles to lowercase
    df['clean'] = df['clean'].map(lambda x: x.lower())
    
    df['clean'] = df['clean'].apply(word_tokenize)
    
    df['clean'] = df['clean'].apply(lambda x: [item for item in x if item.isalpha()])
    
    df['text_clean'] = df['clean'].apply(lambda x: ' '.join(x))
    
    return df.drop(["clean"], axis=1)

In [5]:
df_clean = clean_df(train, "Comment")

In [7]:
df_clean.head(8)

,Insult,Date,Comment,text_clean
0,1,20120618192155Z,"""You fuck your dad.""",you fuck your dad
1,0,20120528192215Z,"""i really don't understand your point.\xa0 It ...",i really do understand your it seems that you ...
2,0,NaN,"""A\\xc2\\xa0majority of Canadians can and has ...",of canadians can and has been wrong before now...
3,0,NaN,"""listen if you dont wanna get married to a man...",listen if you dont wan na get married to a man...
4,0,20120619094753Z,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...",giang ta khi sau tranh con
5,0,20120620171226Z,"""@SDL OK, but I would hope they'd sign him to ...",ok but i would hope they sign him to a contrac...
6,0,20120503012628Z,"""Yeah and where are you now?""",yeah and where are you now
7,1,NaN,"""shut the fuck up. you and the rest of your fa...",shut the fuck up you and the rest of your fagg...


In [13]:
df_clean1 = df_clean.drop(["Date", "Comment"], axis=1)

In [14]:
df_clean1

,Insult,text_clean
0,1,you fuck your dad
1,0,i really do understand your it seems that you ...
2,0,of canadians can and has been wrong before now...
3,0,listen if you dont wan na get married to a man...
4,0,giang ta khi sau tranh con
...,...,...
3942,1,you are both morons and that is never happening
3943,0,many toolbars include spell check like yahoo f...
3944,0,falls i told my boy he should call around toda...
3945,0,how about felix he is sure turning into one he...


In [15]:
df_train, df_test = train_test_split(df_clean1, test_size=0.33, random_state=42)

print(df_train.shape)
print(df_test.shape)

(2644, 2)
(1303, 2)


In [17]:
df_train.to_csv("../data_models/df_train.csv", index=False)
df_test.to_csv("../data_models/df_test.csv", index=False)